In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import io
import requests

# Declare World's ISO Alpha-3 Code value to WRD
world_iso3 = 'WRD'


### Countries to observe

In [ ]:
# Load Countries to observe into DataFrame
country_csv_path = "Resources/Country/Country.csv"
country_df = pd.read_csv(country_csv_path)
# Set World's ISO Alpha-3 Code value to WRD
country_df = country_df.set_index('Country Name')
country_df.loc['World', 'ISO alpha-3 Code'] = world_iso3
country_df = country_df.reset_index()
country_df.head()

### Nation Footprint Data

In [ ]:
# Load National Footprint Data
nf_csv_path = "Resources/NFA/NFA 2018.csv"
nf_df = pd.read_csv(nf_csv_path)

# Set World's ISO Alpha-3 Code value to WRD
nf_df = nf_df.set_index('country')
nf_df.loc['World', 'ISO alpha-3 code'] = world_iso3
nf_df = nf_df.reset_index()

# Filters: 
# records keeping: BiocapTotGHA 
# records filter out: BiocapPerCap, EFConsPerCap/TotGHA, EFExportsPerCap/TotGHA, EFImportsPerCap/TotGHA, EFProdPerCap/TotGHA
nf_df = nf_df[nf_df['record'] == 'BiocapTotGHA']
nf_df.head()

In [ ]:
# Merge National Footprint Data with Countries to observe
merged_nf_df = pd.merge(country_df, nf_df, how='left', left_on='ISO alpha-3 Code', right_on='ISO alpha-3 code')
merged_nf_df.head()

### Food and Agriculture Organization of the UN Data

In [ ]:
# Set FAO stardand url parametrs
fao_area = 'area=' + '%2C'.join(country_df['GFN Country Code'].apply(str))
fao_year = '&year=' + '%2C'.join(np.arange(1995,2019).astype(str))
fao_area_cs = '&area_cs=FAO'
fao_item_cs = '&item_cs=FAO'
fao_tail = '&show_codes=true&show_unit=true&show_flags=true&null_values=false&output_type=csv'

### Production: Livestock Primary

In [ ]:
# Production: Livestock Primary
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/QL?'
element_str = '2313%2C2510'
fao_element = '&element=' + element_str
item_str = '1765'
fao_item = '&item=' + item_str
# finalize fao url
pr_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Get Production: Livestock Primary Data from FOA
response = requests.get(pr_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_pr_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_pr_df.head()


### Trade : Livestock Products

In [ ]:
# Trade: Livestock Products (Import/Export)
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/TP?'
element_str = '2910%2C2610'
fao_element = '&element=' + element_str
item_str = '2077'
fao_item = '&item=' + item_str
# finalize fao url
tr_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Trade: Livestock Products Data from FOA
response = requests.get(tr_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_tr_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_tr_df.head()

### Food Balance: Commodity Balances Livestocks

In [ ]:
# Food Balance: Commodity Balances Livestocks (Domestic Supply Qty)
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/BL?'
element_str = '2300'
fao_element = '&element=' + element_str
item_str = '2943'
fao_item = '&item=' + item_str
# finalize fao url
cb_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Get Food Balance: Commodity Balances Livestocks Data from FOA
response = requests.get(cb_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_cb_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_cb_df.head()


### Investment: Government Expenditure

In [ ]:
# Investment: Government Expenditure (Subsidies)
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/IG?'
element_str = '6110'
fao_element = '&element=' + element_str
item_str = '23164%2C23134'
fao_item = '&item=' + item_str
# finalize fao url
fb_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Get Investment: Government Expenditure Data from FOA
response = requests.get(fb_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_fb_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_fb_df.head()


### OECD: Meat Consumption

In [ ]:
# Load FAO Meat Consumption Data (downloaded from OECD)
fao_mc_csv_path = "Resources/FAO/daily-protein-supply-from-animal-and-plant-based-foods.csv"
fao_mc_df = pd.read_csv(fao_mc_csv_path)

merged_fao_mc_df = pd.merge(country_df, fao_mc_df, how='left', left_on='ISO alpha-3 Code', right_on='Code')
merged_fao_mc_df.head()